In [105]:
import urllib
import urllib3
import bs4
import pickle
import nltk
urllib3.disable_warnings()
from pycorenlp import StanfordCoreNLP
import pandas as pd

def make_soup(myurl):
    pm = urllib3.PoolManager()
    html = pm.urlopen(url = myurl, method = 'GET', redirect= False).data
    return bs4.BeautifulSoup(html, 'html5lib')

def get_text_news(url, tag = 'p'):
    soup = make_soup(url)
    text = ' '.join([t.text.encode('utf-8', 'ignore').decode('utf-8') for t in soup.find_all('p')])
    #print(url)
    return text

def valid_url(url):
    a=urllib.parse.urlsplit(url)
    valid = urllib.parse.urlunparse((a.scheme, a.netloc, urllib.parse.quote(a.path.encode('utf-8')), '', '', ''))
    return valid

def get_entities(text, nlp):
    return nlp.annotate(text, properties={'annotators': 'ner', 'outputFormat': 'json'})

def get_alltext_entities(list_of_links):
    annotation_list = [get_entities(get_text_news(valid_url(url)), nlp) for url in list_of_links]
    return annotation_list

def get_entity_tup(annotator_dict):
    ners = []
    for sentence in annotator_dict['sentences']:
        for entity in sentence['entitymentions']:
            ners.append((entity['ner'],entity['text']))
    return ners

In [2]:
#Creating a StanfordCoreNLP object to comunicate with server in aws instance
nlp = StanfordCoreNLP('http://localhost:9000/')

In [5]:
links = pd.read_csv('links.csv')

In [18]:
links = list(links['0'])

In [107]:
tagged_news = get_alltext_entities(links)

In [120]:
tagged_news[0]['sentences']

list

In [124]:
set([type(article) for article in tagged_news])

{str, dict}

In [125]:
strings = [article for article in tagged_news if type(article)==str]

In [128]:
tagged_news_good = [article for article in tagged_news if type(article)!=str]

In [132]:
entities_words = []
for article in tagged_news_good:
    entities_word += get_entity_tup(article)

In [136]:
entities_word

[('NUMBER', 'dos'),
 ('NUMBER', 'Seis'),
 ('ORGANIZATION', 'Fuerza Única Jalisco'),
 ('ORGANIZATION', 'FUJ'),
 ('NUMBER', 'tres'),
 ('CITY', 'La Huerta'),
 ('NUMBER', '255'),
 ('CITY', 'Guadalajara'),
 ('TITLE', 'fiscal general'),
 ('PERSON', 'Marisela Gómez Cobos'),
 ('CAUSE_OF_DEATH', 'ataque'),
 ('NUMBER', 'cuatro'),
 ('ORGANIZATION', 'FUJ'),
 ('CITY', 'La Huerta'),
 ('NUMBER', 'dos'),
 ('ORGANIZATION', 'pick-up'),
 ('ORGANIZATION', 'Jeep Wrangler'),
 ('CAUSE_OF_DEATH', 'ataque'),
 ('NUMBER', 'dos'),
 ('NUMBER', '80'),
 ('PERSON', 'Villa Purificación'),
 ('PERSON', 'Nemesio Oseguera Cervantes'),
 ('LOCATION', 'Mencho'),
 ('TITLE', 'líder'),
 ('ORGANIZATION', 'Jalisco Nueva'),
 ('NUMBER', '200'),
 ('PERSON', 'José María More'),
 ('CITY', 'Tomatlán'),
 ('CAUSE_OF_DEATH', 'fuego'),
 ('NUMBER', 'tres'),
 ('CITY', 'Puerto Vallarta'),
 ('CITY', 'Manzanillo'),
 ('PERSON', 'Gómez Cobos'),
 ('ORGANIZATION', 'FUJ'),
 ('TITLE', 'helicóptero'),
 ('TITLE', 'policía'),
 ('CITY', 'Guadalajara'),
 

In [137]:
def dict_ent_mentions():
    ent_words_dict = {}
    for tup in entities_word:
        if tup[0] not in ent_words_dict.keys():
            ent_words_dict[tup[0]] = [tup[1]]
        else:
            ent_words_dict[tup[0]].append(tup[1])

In [141]:
ent_words_dict.keys()

dict_keys(['NUMBER', 'ORGANIZATION', 'CITY', 'TITLE', 'PERSON', 'CAUSE_OF_DEATH', 'LOCATION', 'DATE', 'MISC', 'CRIMINAL_CHARGE', 'COUNTRY', 'STATE_OR_PROVINCE', 'NATIONALITY', 'IDEOLOGY', 'RELIGION', 'MONEY', 'PERCENT', 'DURATION', 'ORDINAL'])

In [142]:
for key in ent_words_dict:
    print(key, len(set(ent_words_dict[key])))

NUMBER 2306
ORGANIZATION 10668
CITY 1413
TITLE 743
PERSON 15996
CAUSE_OF_DEATH 169
LOCATION 3306
DATE 1523
MISC 3745
CRIMINAL_CHARGE 78
COUNTRY 184
STATE_OR_PROVINCE 38
NATIONALITY 101
IDEOLOGY 100
RELIGION 67
MONEY 178
PERCENT 268
DURATION 3
ORDINAL 2


In [155]:
set([w.lower() for w in ent_words_dict['CRIMINAL_CHARGE']])

{'abandono',
 'abuso infantil',
 'abuso sexual',
 'adulterio',
 'armas de fuego',
 'asalto',
 'asalto sexual',
 'asesinato',
 'asesinato múltiple',
 'asesino en serie',
 'atentado suicida',
 'bombardeo',
 'calumnia',
 'chantaje',
 'conspiración',
 'conspiración para asesinar',
 'contrabando',
 'crimen organizado',
 'crímenes contra la humanidad',
 'crímenes de guerra',
 'crímenes de odio',
 'cultivo de drogas',
 'delito',
 'delito de tortura',
 'delitos',
 'delitos homicidio',
 'doble asesinato',
 'drogas',
 'drogas lavado de dinero',
 'el tráfico de drogas',
 'espionaje',
 'explosivos',
 'extorsión',
 'fraude',
 'genocidio',
 'homicidio',
 'homicidio involuntario',
 'hurto',
 'incendio provocado',
 'infanticidio',
 'información privilegiada',
 'intimidación',
 'invasión',
 'las armas de fuego',
 'lavado de dinero',
 'malversación',
 'perjurio',
 'pornografía infantil',
 'posesión de armas',
 'prostitución',
 'robo a mano armada',
 'secuestro',
 'soborno',
 'tentativa de asesinato',
 '

In [157]:
cd_words = ['abuso sexual',
 'armas de fuego',
 'asalto',
 'asalto sexual',
 'asesinato',
 'asesinato múltiple',
 'asesino en serie',
 'crimen organizado',
 'crímenes de odio',
 'cultivo de drogas',
 'delitos homicidio',
 'doble asesinato',
 'drogas',
 'drogas lavado de dinero',
 'el tráfico de drogas',
            'extorsión',
 'homicidio',
 'homicidio involuntario',
 'infanticidio',
 'las armas de fuego',
 'lavado de dinero',
 'posesión de armas',
 'robo a mano armada',
 'secuestro',
 'tentativa de asesinato',
 'tortura',
 'traficante de drogas',
 'traición',
 'triple asesinato',
 'tráfico de drogas',
 'violencia doméstica']

In [156]:
set([w.lower() for w in ent_words_dict['CAUSE_OF_DEATH']])

{'accidente',
 'accidente aéreo',
 'accidente de tráfico',
 'accidentes',
 'accidentes automovilísticos',
 'accidentes de tráfico',
 'alcoholismo',
 'aneurisma',
 'angina de pecho',
 'apendicitis',
 'asesinado',
 'asesinato ritual',
 'asfixia',
 'asma',
 'ataque',
 'ataque terrorista',
 'ataques',
 'ataques terroristas',
 'barbitúrico',
 'bronquitis',
 'causas naturales',
 'caídas',
 'choque',
 'cianuro',
 'cirrosis',
 'cirugía',
 'complicaciones',
 'cáncer',
 'cáncer de cabeza y cuello',
 'cáncer de mama',
 'cáncer de piel',
 'cáncer de pulmón',
 'cáncer de páncreas',
 'cáncer de vejiga',
 'cáncer mesotelioma',
 'decapitación',
 'demencia',
 'desastre',
 'desastre natural',
 'desastres',
 'desastres naturales',
 'desnutrición',
 'diabetes',
 'difteria',
 'disparo',
 'duelo',
 'duelos',
 'edema pulmonar',
 'ejecuciones',
 'ejecución',
 'ejecutado',
 'embolia pulmonar',
 'enfermedad',
 'enfermedad cardiovascular',
 'enfermedad pulmonar obstructiva crónica',
 'enfermedad respiratoria',
 

In [158]:
cc_words = ['alcoholismo',
 'asesinado',
 'ataque',
 'ataques',
 'decapitación',
 'disparo',
 'ejecuciones',
 'ejecución',
 'ejecutado',
 'explosiones',
 'explosión',
 'fuego',
 'violencia']

In [172]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("spanish")
from nltk.corpus import stopwords

def stemmed_list(list_of_words):
    temp_list = []
    for word in list_of_words:
        tokens = nltk.word_tokenize(word)
        clean = [stemmer.stem(token.lower()) for token in tokens]
        clean = [token for token in tokens if token not in stopwords.words('spanish')]
        temp_list.append(' '.join(clean))
    return list(set(temp_list))

In [173]:
stemmed_list(cc_words)

['ataque',
 'alcoholismo',
 'decapitación',
 'asesinado',
 'ejecutado',
 'ejecución',
 'explosiones',
 'fuego',
 'ejecuciones',
 'violencia',
 'ataques',
 'explosión',
 'disparo']

In [174]:
stemmed_list(cd_words)

['homicidio',
 'extorsión',
 'delitos homicidio',
 'robo mano armada',
 'tortura',
 'lavado dinero',
 'drogas',
 'crímenes odio',
 'asalto',
 'homicidio involuntario',
 'secuestro',
 'violencia doméstica',
 'tentativa asesinato',
 'drogas lavado dinero',
 'traición',
 'asesinato',
 'abuso sexual',
 'asalto sexual',
 'asesinato múltiple',
 'infanticidio',
 'tráfico drogas',
 'cultivo drogas',
 'asesino serie',
 'armas fuego',
 'doble asesinato',
 'crimen organizado',
 'posesión armas',
 'traficante drogas',
 'triple asesinato']

In [175]:
set([w.lower() for w in ent_words_dict['COUNTRY']])

{'afganistán',
 'alba',
 'alemania',
 'america',
 'andorra',
 'angola',
 'aotearoa nueva zelanda',
 'arabia saudita',
 'argelia',
 'argentina',
 'armenia',
 'australia',
 'austria',
 'azerbaiyán',
 'aztlán',
 'bahamas',
 'bahrein',
 'bangladesh',
 'belice',
 'birmania',
 'bohemia',
 'bolivia',
 'bosnia y herzegovina',
 'brasil',
 'bulgaria',
 'bélgica',
 'camboya',
 'camerún',
 'canadá',
 'chile',
 'china',
 'chipre',
 'colombia',
 'columbia',
 'corea',
 'costa de marfil',
 'costa rica',
 'croacia',
 'cuba',
 'dinamarca',
 'eau',
 'ecuador',
 'eeuu',
 'egipto',
 'el salvador',
 'emiratos árabes unidos',
 'eritrea',
 'escocia',
 'eslovaquia',
 'eslovenia',
 'espanya',
 'españa',
 'estado de israel',
 'estado español',
 'estados unidos',
 'estonia',
 'etiopía',
 'federación rusa',
 'filipinas',
 'finlandia',
 'francia',
 'gabón',
 'gambia',
 'georgia',
 'gibraltar',
 'gran bretaña',
 'granada',
 'grecia',
 'guatemala',
 'guyana',
 'haití',
 'holanda',
 'honduras',
 'hong kong',
 'hungría

In [177]:
set([w.lower() for w in ent_words_dict['CITY']])

{'cholula',
 'chisinau',
 'medicina',
 'tabasco',
 'sonsonate',
 'aldama',
 'mijas',
 'montemorelos',
 'san juan bautista tuxtepec',
 'sébaco',
 'lagos',
 'arcadia',
 'dhule',
 'usulután',
 'mapastepec',
 'zaachila',
 'san pablo tecalco',
 'sana',
 'altos',
 'la habana',
 'dublín',
 'pisco',
 'arena',
 'pharr',
 'tamiahua',
 'gwadar',
 'talavera',
 'chilapa',
 'zihuatanejo',
 'ambón',
 'eibar',
 'el socorro',
 'la esperanza',
 'bada huesca',
 'catemaco',
 'xoxocotla',
 'salina cruz',
 'atlanta',
 'lista',
 'susa',
 'domingo',
 'santa bárbara',
 'san lucas',
 'lota',
 'campanilla',
 'tonila',
 'alejandría',
 'atenas',
 'dolores hidalgo',
 'san lorenzo de el escorial',
 'la línea',
 'las palmas',
 'salt lake city',
 'xalapa',
 'murcia',
 'charlotte',
 'washington dc',
 'baja',
 'cuernavaca',
 'san calixto',
 'nueva esperanza el limón san antonio san benito',
 'nueva italia',
 'cádiz',
 'angol',
 'río blanco',
 'puente grande',
 'penjamillo',
 'chicxulub',
 'caba',
 'san josé el alto',
 '